In [1]:
#start

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9,application_1717118893495_0011,pyspark,idle,Link,Link,user_n-renukumar_rajendran_draup_com,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.addPyFile("s3://jobs-emr-etl-config/jobs_run/prod/cc/ML_ETL_Config.py")
from ML_ETL_Config import *
utils_module = MlEtlUtils(spark, sc)
to_import = [name for name in dir(utils_module) if not (name.startswith('_') | (name=="spark") | (name=="sc"))]
globals().update({name: getattr(utils_module, name) for name in to_import})
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

404
404
/usr/local/lib/python3.7/site-packages/urllib3/connectionpool.py:1068: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/usr/local/lib/python3.7/site-packages/urllib3/connectionpool.py:1068: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,

In [3]:
#sc.install_pypi_package("tldextract")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#import tldextract
import re
import string

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
linkedin=spark.read.format('delta').load('s3://etl-datahouse/prod/dcd/xverum/delta/').selectExpr('account_name as linkedin_Name','social_url as Linkedin_link','website as Linkedin_website','permalink','industry','headquarter','about','company_size','employee_count','year_established','logo_url','account_type','date_updated')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
linkedin.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

49108470

In [8]:
linkedin.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- linkedin_Name: string (nullable = true)
 |-- Linkedin_link: string (nullable = true)
 |-- Linkedin_website: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- industry: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- headquarter: string (nullable = true)
 |-- about: string (nullable = true)
 |-- company_size: string (nullable = true)
 |-- employee_count: long (nullable = true)
 |-- year_established: string (nullable = true)
 |-- logo_url: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- date_updated: string (nullable = true)

In [7]:
df1=linkedin.withColumn('linkedin_lower_name',lower(col('linkedin_Name')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
linkedin[["Linkedin_website"]].show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------+
|Linkedin_website                                                                          |
+------------------------------------------------------------------------------------------+
|https://www.abetterwaytherapy.com/                                                        |
|https://www.facebook.com/pages/biz/beauty_salon-leusden-3831%20HN/T-begin-344504566037640/|
|http://werkstatt-essingen.de                                                              |
|null                                                                                      |
|null                                                                                      |
|http://10xceocoaching.com                                                                 |
|null                                                                                      |
|http://www.100rota.pt                                                

# Clean Name

In [9]:
#name clean
def clean_name(name):
    # Remove punctuation characters
    name = re.sub(f"[{re.escape(string.punctuation)}]", '', name)
    # Remove leading and trailing spaces
    name = name.strip()
    return name

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
clean_name_udf = udf(clean_name, StringType())
df2 = df1.withColumn("clean_name",clean_name_udf(df1["linkedin_lower_name"])).drop('linkedin_lower_name')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df2[["clean_name"]].show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------+
|clean_name                               |
+-----------------------------------------+
|a better way therapy                     |
|t begin                                  |
|diewerkstatt kunst und kurse             |
|consiglio daniel headley anna            |
|10 north walnut inc                      |
|10x ceo coaching llc                     |
|1 third avenue management company limited|
|100 rota                                 |
|101 realty group inc                     |
|19 rivoli                                |
|1four1 payroll 023 ltd                   |
|18 queens road residents limited         |
|1876 heritage inn                        |
|1896 homestead                           |
|2 business one                           |
|1st class transporting llc               |
|1st wendover scouts                      |
|1d accounts ltd                          |
|shopify plus experts                     |
|1st service uk process servers 

In [12]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re


def generalize_company_name(name):
    name = str(name).lower()
    name = re.sub(r'\([^)]*\)', '', name)
    # end_char = r'[^\w\s]+$'
    name = re.sub(r'[^\w\s]+$', '', name)
    # ms_pattern = r'^m/s[^\w\s]*\s*'
    name = re.sub(r'^m/s[^\w\s]*\s*', '', name)
    name = re.sub(' +', ' ', name)
    name = name.strip()
    common_terms = [
        'PRIVATE.', 'PVT.', 'PVT. LTD.', 'PVT. LTD', 'CO.', 'CORPORATION',
        'CORP', 'INC.', 'INC', 'LLC', 'LLLP.', 'LIMITED LIABILITY', 'L.L.C.',
        'L.L.P.', 'L.P.', 'L.L.L.P.', 'L.L.L.P', 'LTD.', 'LIMITED.',
        'PRIVATE LIMITED.', 'PVT LTD.', 'PVT. LTD.', 'PVT. LTD', 'INDIA.',
        'PRIVATE', 'PVT', 'LTD', 'LIMITED', 'PRIVATE LIMITED', 'PVT LTD', 'CO',
        'LLP', 'LP', 'LLLP', '&', '\.', '\(\)', '(P)', 'Corporates', 'Corporate'
        'SOLUTIONS', 'SOLUTION', 'P.', 'P', '\.', 'india.', 'india',
        'Liabili Ty', 'Servic Es', 'Limi Ted', 'Private.Limited.', 'Limted',
        'Lim Ited', 'Lmited', 'Limted', 'L Imited', 'Liability', 'L iability',
        'Li ability', 'Lia bility', 'Liab ility', 'Liabi lity', 'Liabil ity',
        'Liabilit y', 'L imited', 'Lim ited', 'Limit ed', 'Limite d',
        'Services', 'Service', 'PARTNERSHIPS', 'PARTNERSHIP', 'TECHNOLOGIES',
        'GROUP', 'TECHNOLOGIE', 'TECHNOLOGY', 'Infotech'
    ]
    for term in common_terms:
        og_name = name
        name = re.sub(rf'(?i)\s{term}$', '', name, flags=re.IGNORECASE).strip()
        if og_name != name:
            name = generalize_company_name(name)

    name = re.sub(' +', ' ', name).replace(" ","")
    return name


# Register the UDF with Spark
clean_name_udf = udf(generalize_company_name, StringType())

df3 = df2.withColumn('cleaned_name', clean_name_udf(df2['clean_name'])).drop('clean_name')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df3[["cleaned_name"]].show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+
|cleaned_name                 |
+-----------------------------+
|abetterwaytherapy            |
|tbegin                       |
|diewerkstattkunstundkurse    |
|consigliodanielheadleyanna   |
|10northwalnut                |
|10xceocoaching               |
|1thirdavenuemanagementcompany|
|100rota                      |
|101realty                    |
|19rivoli                     |
|1four1payroll023             |
|18queensroadresidents        |
|1876heritageinn              |
|1896homestead                |
|2businessone                 |
|1stclasstransporting         |
|1stwendoverscouts            |
|1daccounts                   |
|shopifyplusexperts           |
|1stserviceukprocessservers   |
+-----------------------------+
only showing top 20 rows

# Clean website

In [14]:
def companys(link):
    if link:
        match = re.search(r"https?://(?:www\.)?([^/.]+)", link)
        if match:
            return match.group(1)
        elif '.com' in link:
            return link.split('.com')[0].split('.')[-1]
    return None


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
domain_name_udf = udf(companys, StringType())
final_df = df3.withColumn("cleaned_domain", domain_name_udf(df3['Linkedin_website']))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
final_df.select("cleaned_domain").show(truncate = False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|cleaned_domain    |
+------------------+
|abetterwaytherapy |
|facebook          |
|werkstatt-essingen|
|null              |
|null              |
|10xceocoaching    |
|null              |
|100rota           |
|null              |
|19rivoli          |
|null              |
|null              |
|1876heritageinn   |
|null              |
|cap               |
|null              |
|1stwendoverscouts |
|null              |
|kondasoft         |
|null              |
+------------------+
only showing top 20 rows

In [17]:
Nall_df = final_df.filter(col("cleaned_name").isNotNull() & col("cleaned_domain").isNotNull()).drop_duplicates()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
Nall_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- linkedin_Name: string (nullable = true)
 |-- Linkedin_link: string (nullable = true)
 |-- Linkedin_website: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- industry: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- headquarter: string (nullable = true)
 |-- about: string (nullable = true)
 |-- company_size: string (nullable = true)
 |-- employee_count: long (nullable = true)
 |-- year_established: string (nullable = true)
 |-- logo_url: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- date_updated: string (nullable = true)
 |-- cleaned_name: string (nullable = true)
 |-- cleaned_domain: string (nullable = true)

In [19]:
final_df1 = Nall_df.filter(col("cleaned_name") == col("cleaned_domain"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#final_df1.write.mode("overwrite").parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/temp/xverum_name_website_maping')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
final_df2 = spark.read.parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/temp/xverum_name_website_maping')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
final_df2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- linkedin_Name: string (nullable = true)
 |-- Linkedin_link: string (nullable = true)
 |-- Linkedin_website: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- industry: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- headquarter: string (nullable = true)
 |-- about: string (nullable = true)
 |-- company_size: string (nullable = true)
 |-- employee_count: long (nullable = true)
 |-- year_established: string (nullable = true)
 |-- logo_url: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- date_updated: string (nullable = true)
 |-- cleaned_name: string (nullable = true)
 |-- cleaned_domain: string (nullable = true)

In [28]:
final_df2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8476045

In [29]:
final_df2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------+----------------+--------------------+--------------+------------+-----------------+-----------------+
|       linkedin_Name|       Linkedin_link|    Linkedin_website|           permalink|            industry|         headquarter|               about|company_size|employee_count|year_established|            logo_url|  account_type|date_updated|     cleaned_name|   cleaned_domain|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------+----------------+--------------------+--------------+------------+-----------------+-----------------+
|Bangalore Ceramic...|https://www.linke...|http://bangalorec...|bangalore-ceramic...|                  []|Bangalore, Karnataka|                null|        null|  

In [30]:
split_ratios = [1/9] * 9  # Equal split ratios for 9 parts
split_datasets = final_df2.randomSplit(split_ratios, seed=42)

# Access each split DataFrame
split_df1 = split_datasets[0]
split_df2 = split_datasets[1]
split_df3 = split_datasets[2]
split_df4 = split_datasets[3]
split_df5 = split_datasets[4]
split_df6 = split_datasets[5]
split_df7 = split_datasets[6]
split_df8 = split_datasets[7]
split_df9 = split_datasets[8]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
split_df1.count() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

941524

In [33]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df1.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Xverum_data_dump_set1_31-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'JBX3E95N6DEJ95VH', 'HostId': 'LB+77A1kDZWMukkr5Og8ao2HWjPXIfSSTBiigySwnbty4fCfNvO7aZQ01l1E6kIc0ZD25pX0O/AMzdUxsDzkdw==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'LB+77A1kDZWMukkr5Og8ao2HWjPXIfSSTBiigySwnbty4fCfNvO7aZQ01l1E6kIc0ZD25pX0O/AMzdUxsDzkdw==', 'x-amz-request-id': 'JBX3E95N6DEJ95VH', 'date': 'Fri, 31 May 2024 04:57:55 GMT', 'x-amz-expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"8ba31d7b8d3ee34a529080f45b4eb2b8"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"8ba31d7b8d3ee34a529080f45b4eb2b8"', 'ServerSideEncryption': 'AES256'}

In [34]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df2.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Xverum_data_dump_set2_31-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'EHBX4YVKH9KMMDP4', 'HostId': 'usKk5vp52FINjkBHqsMQrptv1xaayD7NvRoo8g+sHISuJZvifzkNkt3YmjFQT780MTGoatloByDUG5m2VtOUUQ==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'usKk5vp52FINjkBHqsMQrptv1xaayD7NvRoo8g+sHISuJZvifzkNkt3YmjFQT780MTGoatloByDUG5m2VtOUUQ==', 'x-amz-request-id': 'EHBX4YVKH9KMMDP4', 'date': 'Fri, 31 May 2024 04:59:40 GMT', 'x-amz-expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"417ccb480832382db6e9d25f12b61416"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"417ccb480832382db6e9d25f12b61416"', 'ServerSideEncryption': 'AES256'}

In [35]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df3.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Xverum_data_dump_set3_31-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'SF780D64QCJ6D19D', 'HostId': '59l2zjrV/6fqvpxy+tqNyuyg5x4l0weUdAkHvI9htTr7fYkmC5V5FUNcICum6LajE7Jrx4nxRQc=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '59l2zjrV/6fqvpxy+tqNyuyg5x4l0weUdAkHvI9htTr7fYkmC5V5FUNcICum6LajE7Jrx4nxRQc=', 'x-amz-request-id': 'SF780D64QCJ6D19D', 'date': 'Fri, 31 May 2024 05:23:53 GMT', 'x-amz-expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"57b1e63b2c73fde38a260bfd9c52fa54"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"57b1e63b2c73fde38a260bfd9c52fa54"', 'ServerSideEncryption': 'AES256'}

In [36]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df4.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Xverum_data_dump_set4_31-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'NNECVW764ZF4N99Y', 'HostId': 'thpxbRg9Bnxgxq6uiD82VGiqE+BIW+Zpq6Ceqej2JGPSp7AMcfL82Ik95lK0CDBvVRsWR6DYkh8=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'thpxbRg9Bnxgxq6uiD82VGiqE+BIW+Zpq6Ceqej2JGPSp7AMcfL82Ik95lK0CDBvVRsWR6DYkh8=', 'x-amz-request-id': 'NNECVW764ZF4N99Y', 'date': 'Fri, 31 May 2024 05:25:36 GMT', 'x-amz-expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"754f443926a990564998e5f68e465b9a"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"754f443926a990564998e5f68e465b9a"', 'ServerSideEncryption': 'AES256'}

In [37]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df5.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Xverum_data_dump_set5_31-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'Z7MGTGKEP7BNANKB', 'HostId': 'UFkKLJpXtEozsocFhtR7H6P1G+tT1YsgdQPcED6dp7J2S1/dBHaTSwRC8FwDX3HrxrAqhB5h4+/PR3PjtEe3XFGsP+I6M+1qaZvw0tv0dSE=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'UFkKLJpXtEozsocFhtR7H6P1G+tT1YsgdQPcED6dp7J2S1/dBHaTSwRC8FwDX3HrxrAqhB5h4+/PR3PjtEe3XFGsP+I6M+1qaZvw0tv0dSE=', 'x-amz-request-id': 'Z7MGTGKEP7BNANKB', 'date': 'Fri, 31 May 2024 05:27:31 GMT', 'x-amz-expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"ddc04ab68b195f84adb1d5e4bfedca30"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"ddc04ab68b195f84adb1d5e4bfedca30"', 'ServerSideEncryption': 'AES256'}

In [38]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df6.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Xverum_data_dump_set6_31-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '274RA4ZEYKKFD2QG', 'HostId': 'nzlvQA0DeVoq2gJJTMgce6nqQ8BMWPGrJtQpaDZEf4tDo9pwoqzuOWRsMD1UuedwYFCI1ZPhW8i+xQu0LJ0Nf1N/0b6WGyOMpGZXEux81/c=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'nzlvQA0DeVoq2gJJTMgce6nqQ8BMWPGrJtQpaDZEf4tDo9pwoqzuOWRsMD1UuedwYFCI1ZPhW8i+xQu0LJ0Nf1N/0b6WGyOMpGZXEux81/c=', 'x-amz-request-id': '274RA4ZEYKKFD2QG', 'date': 'Fri, 31 May 2024 05:29:59 GMT', 'x-amz-expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"d194ace671c8465bc618c7757987ca29"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"d194ace671c8465bc618c7757987ca29"', 'ServerSideEncryption': 'AES256'}

In [40]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df7.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Xverum_data_dump_set7_31-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '7401QDHYQSR1PE15', 'HostId': '8TGkEn88B4HAAzCnHV5uioaXzmLrEkyLs+iRQhnyE/Fb42hEhyD1XqlTVmpRb4gxhVyOR5WkM4Q=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '8TGkEn88B4HAAzCnHV5uioaXzmLrEkyLs+iRQhnyE/Fb42hEhyD1XqlTVmpRb4gxhVyOR5WkM4Q=', 'x-amz-request-id': '7401QDHYQSR1PE15', 'date': 'Fri, 31 May 2024 05:32:26 GMT', 'x-amz-expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"f5d54255c1378bc3b04a0c47ad71e6af"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"f5d54255c1378bc3b04a0c47ad71e6af"', 'ServerSideEncryption': 'AES256'}

In [41]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df8.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Xverum_data_dump_set8_31-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'PFW0HBY4DWFG7CDC', 'HostId': 'UbNOIiq874woVUi9BTISEKtqTRS6i4Ve9mr0w2FO71gLwCH2KmpuAJ8HSHt12HAR0UNrmvuCtrA=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'UbNOIiq874woVUi9BTISEKtqTRS6i4Ve9mr0w2FO71gLwCH2KmpuAJ8HSHt12HAR0UNrmvuCtrA=', 'x-amz-request-id': 'PFW0HBY4DWFG7CDC', 'date': 'Fri, 31 May 2024 05:34:54 GMT', 'x-amz-expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"c8f9dc5158761cab58197b75ce3fa07f"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"c8f9dc5158761cab58197b75ce3fa07f"', 'ServerSideEncryption': 'AES256'}

In [42]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=split_df9.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Xverum_data_dump_set9_31-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'B5M2W1G406KG47MV', 'HostId': '4Btua29XwMiziDq1ilodAuLgy98Fbvt/Obz1A6F8BO7OJtZw0CKJHj0SyxMC/QnmVco8KWKweak5baDBn+JExg==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '4Btua29XwMiziDq1ilodAuLgy98Fbvt/Obz1A6F8BO7OJtZw0CKJHj0SyxMC/QnmVco8KWKweak5baDBn+JExg==', 'x-amz-request-id': 'B5M2W1G406KG47MV', 'date': 'Fri, 31 May 2024 05:36:27 GMT', 'x-amz-expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"51e2419437c5813d72043102273be4c8"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 06 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"51e2419437c5813d72043102273be4c8"', 'ServerSideEncryption': 'AES256'}